In [4]:
class Node():
    '''
    This class is to be used by DAG, it represetns a single node in a graph
    '''
    def __init__(self,inEdges,outEdges, name,score):
        '''
        :param name
        :param inEdges: list of input edges to a node with thier wieghts. Each edge represented as a list with two elements [prevNode,wieght]
        :param outEdges: same as inEdges but for the output edges. Each element is [nextNode, wieght]
        :param name: int that names the node
        :param score: int that holds the maximum score of the node, calculated by the weights of the largest path to this node
        '''
        self.inEdges = inEdges
        self.outEdges = outEdges
        # bool array that keeps track of if the edge has been traversed.
        # Index of 1 or 0 in inEdgesReady corresponds to the edge in inEdges at the index.
        # 1 means the edge hasn't been traversed, 0 means it has
        self.inEdgesReady = []
        self.outEdgesReady = []
        self.name = name
        self.score = score

class DAG():
    '''
    Houses all functions that find the longest path in a DAG.
    The DAG is represented by a dictionary of nodes, with key value pairs corresponding to the name of the node and the node {name:Node}
    '''
    def __init__(self, dirGraph, source, sink):
        self.nodes = self.genNodes(dirGraph)
        self.source = source
        self.sink = sink
        self.nodes[self.source].score = 0

        # This block removes all nodes and thier edges if they cannot be found from the source node
        falseStarts = self.findFalseStarts()
        while len(falseStarts) > 0: # exits when all nodes have inEdges other than the source node
            for falseStart in falseStarts:
                for node in self.nodes.values():
                    for edge in node.inEdges:
                        if edge[0] == falseStart:
                            node.inEdges.pop(node.inEdges.index(edge))
                del self.nodes[falseStart]
            falseStarts = self.findFalseStarts() # finds the next layer of nodes with no input edges

        for node in self.nodes.keys():
            self.nodes[node].inEdgesReady = [1 for x in self.nodes[node].inEdges]
            self.nodes[node].outEdgesReady = [1 for x in self.nodes[node].outEdges]


    def findFalseStarts(self):
        '''
        finds all nodes in the DAG that do not have input edges and are not the source node
        :return: list of node names
        '''
        falseStarts = []
        for name, node in self.nodes.items():
            if (len(node.inEdges) == 0) & (name != self.source):
                falseStarts.append(name)
        return falseStarts

    def genNodes(self, dirGraph):
        '''
        turns a directed graph, represented as a dicitonary, into a series of nodes that is sotred in a dictionary with key,value pairs representing the label of each Node with the corresponding Node object as the value
        :param dirGraph: dictionary representing the adjacency list of a graph
        :return: nodes, a dictionary of nodes, where the key is the nodes label and the value is the corresponding Node object
        '''
        nodes = {}
        for node, edges in dirGraph.items():  # iterates through the ajacency list
            if node not in nodes:  # checks to see if the node has been seen yet
                nodes[node] = Node([], dirGraph[node], node, -900000)  # creates a new Node if its not in nodes yet
            else:
                for edge in edges:  # iterates through the output edges
                    nodes[node].outEdges.append(edge)  # adds edge if node already exists
            for edge in edges:  # iterates through the input edges , adds to node if it exists, otherwise it makes a new Node
                if edge[0] not in nodes:
                    nodes[edge[0]] = Node([[node,edge[1]]], [], edge[0], -900000)
                else:
                    nodes[edge[0]].inEdges.append([node,edge[1]])
        return nodes

    def topoOrder(self):
        '''
        Returns a topological order of the DAG, although the returned value is not used.
        The real function of this method is to calculate the scores of each node.
        The score of a node reprentes the maximum value obtained by summing the weights of the path taken to get to that node.
        This score is used in longestPath to find the longest path based on the score of the sink node

        :return: list of all nodes in a topological order
        '''

        
        longest = []  # will hold the longest path
        topolOrder = []
        stack = [self.nodes[self.source]]




        while len(stack) > 0:  # exits when the stack is empty
            toAdd = []

            for i in range(len(stack[-1].outEdges)): # iterates through the nodes connected to out edges of the current node
                currEdge = stack[-1].outEdges[i] # renames the curent edge for clarity
                currNode = self.nodes[currEdge[0]] # renames the current node for clarity

                stack[-1].outEdgesReady[i] = 0 # Changes the ready value of the root node to 0, indicating that it has been traversed

                for inEdge in currNode.inEdges: # looks through the in edges of the current node for the current edge and sets its ready value to 0, indicating it has been traversed
                    if inEdge[0] == stack[-1].name:
                        currNode.inEdgesReady[currNode.inEdges.index(inEdge)] = 0

                if 1 not in currNode.inEdgesReady:
                    toAdd.append(currNode)

                prevScore = stack[-1].score
                newScore = currEdge[1] + prevScore
                if currNode.score < newScore:
                    currNode.score = newScore
            topolOrder.append(stack.pop(-1))

            for node in toAdd: # adds next nodes to stack

                stack.append(node)
        return topolOrder


    def longestPath(self):
        '''
        Given a topological ordering of nodes from a DAG, this function will output the longest path in that ordering

        It starts at the sink of of the DAG and works backward using the scores to find the path
        :param topoOrder:
        :return longestPath: list of nodes
        '''

        longestPath = [self.nodes[self.sink]]
        while longestPath[-1].name != self.source: # exits when the path gets back to the source node
            for edge in longestPath[-1].inEdges: # iterates through inEdges in the current node
                if self.nodes[edge[0]].score + edge[1] == longestPath[-1].score: # moves the curent
                    longestPath.append(self.nodes[edge[0]])
                    break

        return longestPath[::-1]

def main(fName, params=None):
    '''
    Handles input and output, uses DAG() to find the longest path in the input file
    :param fName:
    :param params:
    :return:
    '''

    dirGraph = {}
    if fName == '':
        quit('file not specified')
    else:
        with open(fName) as inFile:
            lines = inFile.readlines()
            source = int(lines[0])
            sink = int(lines[1])
            for line in lines[2:]:  # Reads each line and puts it into a dictionary that represents the adjacency list of the graph
                edges = line.strip().split('->')
                if int(edges[0]) in dirGraph.keys():
                    dirGraph[int(edges[0])].append([int(x) for x in edges[1].split(':')])
                else:
                    dirGraph[int(edges[0])] = [[int(x) for x in edges[1].split(':')]]

    newDAG = DAG(dirGraph,source,sink)
    newDAG.topoOrder()
    longest = newDAG.longestPath()
    print(longest[-1].score)
    toPrint = ''
    for node in longest:
        toPrint += str(node.name) + '->'
    toPrint = toPrint.rstrip('->')
    print(toPrint)

if __name__ == '__main__':
    main(fName='sample.txt')

62
0->14->29->44


In [5]:
main('input')

FileNotFoundError: [Errno 2] No such file or directory: 'input'

This notebook is meant to take in a DAG (Directed Acyclic Graph) represented as an weighted ajacency list, and return the longest path in that DAG.

It has two main classes, DAG() and Node(). The Node object represents a node in the DAG, and the DAG is represetned as a diciotnary of Nodes.

It finds the longest path by finding a topological ordering of the DAG and then uses the scores to work backwards through the DAG to find the longest path.

Returns the score of the sink node along with the longest path. 